In [1]:
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
#lei=law, n=article number, p=paragraph number, inciso=item, alinea=point
class Scrap:
    def __init__(self, lei, n=None, p=None, inciso=None):
        """Initializing the driver"""
        self.lei = lei
        self.n = n
        self.p = p
        
        self.art = str('Art. ' + str(self.n))
        self.lista_paragrafo = []
        self.lista_inciso = []
    
    def conecta(self):
        self.url = 'https://www2.camara.leg.br/busca/?q=' + self.lei
        option = Options()
        option.headless = True
        driver = webdriver.Firefox(options=option)
        driver.get(self.url)
        driver.find_element_by_xpath("//div[@id='resultadoBusca']//ul//li//span//a").click()
        driver.find_element_by_xpath("//div[@class='sessao']//a").click()
        element = driver.find_element_by_xpath("//div[@class='texto']")
        self.titulo = driver.find_element_by_class_name('ementa')
        self.html_content = element.get_attribute('outerHTML')
        self.titulo = self.titulo.get_attribute('outerHTML')
        driver.quit()
        return self.html_content
        
    def title(self):
        """Returns the law title"""
        self.soup = BeautifulSoup(self.html_content, 'lxml')
        self.texto = self.soup.find_all(text=True)
        self.titulo = BeautifulSoup(self.titulo, 'lxml')
        self.titulo = self.titulo.find_all(text=True)
        for i in range(len(self.titulo)):
            self.titulo[i] = self.titulo[i].replace('\n', '')
            self.titulo[i] = self.titulo[i].replace('\t', '')
        return self.titulo[0]
   
    def article(self):
        self.inicio_article = re.search(f'Art. {self.n}', self.html_content).start()
        return self.html_content[self.inicio_article:].replace('&nbsp;', '-').split('<br>')[0]
    
    def paragraph(self):
        inicio_article = re.search(f'Art. {self.n}', self.html_content).start()
        #return inicio_article
        try:
            inicio_paragraph = re.search(f'§ {self.n}º', self.html_content[inicio_article:]).start()
            fim_paragraph = re.search('<br>', self.html_content[(inicio_paragraph+inicio_article):]).start()
            #return[inicio_paragraph, fim_paragraph]
            #return self.inicio_paragraph
            return self.html_content[inicio_paragraph:]
        except:
            return f'deu ruim'

In [4]:
teste = Scrap(lei='lei 1.060', n=4, p=1)
conteudo = teste.conecta()
conteudo

'<div class="texto">\n\t\t\t\t\t\t\t<p>O PRESIDENTE DA REPÚBLICA:&nbsp;<br>Faço saber que o CONGRESSO NACIONAL decreta e eu sanciono a seguinte Lei:&nbsp;<br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Art. 1º&nbsp;Os poderes públicos federal e estadual concederão assistência judiciária aos necessitados nos têrmos da presente Lei. <br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Art. 2º&nbsp;Gozarão dos benefícios desta Lei os nacionais ou estrangeiros residentes no país, que necessitarem recorrer à Justiça penal, civil, militar ou do trabalho. <br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <i>Parágrafo único. </i>Considera-se necessitado, para os fins legais, todo aquêle cuja situação econômica não lhe permita pagar as custas do processo e os honorários de advogado, sem prejuízo do sustento próprio ou da família. <br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Art. 3º&nbsp;A assistência judiciária compreende as seguintes isenções: <br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; I - das taxas judiciárias e dos selos; <br>&nbsp;&nbsp

In [3]:
teste.title()

'Estabelece normas para a concessão de assistência judiciária aos necessitados. '

In [23]:
#usar esta variável para testas o parágrafo
paragrafo = BeautifulSoup(conteudo, 'lxml').find_all()
paragrafo[0]#desta forma é retornado tudo dentro da tag <html>

<html><body><div class="texto">
<p>O PRESIDENTE DA REPÚBLICA: <br/>Faço saber que o CONGRESSO NACIONAL decreta e eu sanciono a seguinte Lei: <br/><br/>     Art. 1º Os poderes públicos federal e estadual concederão assistência judiciária aos necessitados nos têrmos da presente Lei. <br/><br/>     Art. 2º Gozarão dos benefícios desta Lei os nacionais ou estrangeiros residentes no país, que necessitarem recorrer à Justiça penal, civil, militar ou do trabalho. <br/><br/>      <i>Parágrafo único. </i>Considera-se necessitado, para os fins legais, todo aquêle cuja situação econômica não lhe permita pagar as custas do processo e os honorários de advogado, sem prejuízo do sustento próprio ou da família. <br/><br/>     Art. 3º A assistência judiciária compreende as seguintes isenções: <br/><br/>      I - das taxas judiciárias e dos selos; <br/>      II - dos emolumentos e custas devidos aos Juízes, órgãos do Ministério Público e serventuários da Justiça; <br/>      III - das despesas com as pub

In [98]:
str(paragrafo[0]).replace('\xa0', '')
paragrafo2 = str(paragrafo[0]).replace('\xa0', '')

In [120]:
#outra maneira de Buscar o artigo, para poder usar para buscar os parágrafos!
n = 4
inicio_artigo = re.search(f'Art. {n}', paragrafo2).start()
fim_artigo = re.search('<b', paragrafo2[(inicio_artigo):]).start()
#str(paragrafo[0])[inicio_artigo:].replace('\xa0', '').split('<br/>')[0]
fim_artigo += inicio_artigo
paragrafo2[inicio_artigo: fim_artigo]
#Coooooonseguuuuuuiii!!

'Art. 4ºA parte, que pretender gozar os benefícios da assistência judiciária, requererá ao Juiz competente lhos conceda, mencionando, na petição, o rendimento ou vencimento que percebe e os encargos próprios e os da família. '

In [65]:
teste.article()

'Art. 4º-A parte, que pretender gozar os benefícios da assistência judiciária, requererá ao Juiz competente lhos conceda, mencionando, na petição, o rendimento ou vencimento que percebe e os encargos próprios e os da família. '

In [170]:
teste.title()

'Estabelece normas para a concessão de assistência judiciária aos necessitados. '

In [171]:
teste.article()

'Art. 4º-A parte, que pretender gozar os benefícios da assistência judiciária, requererá ao Juiz competente lhos conceda, mencionando, na petição, o rendimento ou vencimento que percebe e os encargos próprios e os da família. '

In [66]:
teste.paragraph()

' advogado e peritos. <br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Art. 4º&nbsp;A parte, que pretender gozar os benefícios da assistência judiciária, requererá ao Juiz competente lhos conceda, mencionando, na petição, o rendimento ou vencimento que percebe e os encargos próprios e os da família. <br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; § 1º A petição será instruída por um atestado de que conste ser o requerente necessitado, não podendo pagar as despesas do processo. Êste documento será expedido, isento de selos e emolumentos, pela autoridade policial ou pelo Prefeito municipal. <br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; § 2º Nas capitais dos Estados e no Distrito Federal, o atestado da competência do Prefeito poderá ser expedido por autoridade expressamente designada pelo mesmo. <br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Art. 5º&nbsp;O Juiz, se não tiver fundadas razões para indeferir o pedido, deverá julgá-lo de plano, motivando ou não o deferimento dentro do prazo de setenta e duas horas. <br><br>&nbs